In [1]:
import sys
!{sys.executable} -m pip install pyfinance

In [8]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [9]:
data = pd.read_excel("isx2010C.xls")

In [10]:
data.head()

,2009-09-21 00:00:00,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570,Unnamed: 48,Unnamed: 49,date
,<int64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<object>
0,86,152.20,NaN,NaN,NaN,132.60,NaN,NaN,NaN,113.20,NaN,NaN,NaN,94.55,NaN,NaN,NaN,76.45,NaN,NaN,NaN,59.25,NaN,NaN,NaN,43.60,NaN,NaN,NaN,29.80,NaN,NaN,NaN,18.65,NaN,NaN,NaN,10.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491.34,0.11,21.9.2009
1,85,155.05,NaN,NaN,NaN,135.35,NaN,NaN,NaN,115.90,NaN,NaN,NaN,96.90,NaN,NaN,NaN,78.75,NaN,NaN,NaN,61.45,NaN,NaN,NaN,45.45,NaN,NaN,NaN,31.40,NaN,NaN,NaN,19.80,NaN,NaN,NaN,11.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494.35,0.11,22.9.2009
2,84,152.40,NaN,NaN,NaN,132.70,NaN,NaN,NaN,113.30,NaN,NaN,NaN,94.35,NaN,NaN,NaN,76.15,NaN,NaN,NaN,59.05,NaN,NaN,NaN,43.40,NaN,NaN,NaN,29.55,NaN,NaN,NaN,18.45,NaN,NaN,NaN,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490.36,0.11,23.9.2009
3,83,147.00,NaN,NaN,NaN,127.25,NaN,NaN,NaN,108.05,NaN,NaN,NaN,89.70,NaN,NaN,NaN,71.95,NaN,NaN,NaN,55.25,NaN,NaN,NaN,39.70,NaN,NaN,NaN,26.95,NaN,NaN,NaN,16.45,NaN,NaN,NaN,8.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,486.99,0.11,24.9.2009
4,82,145.55,NaN,NaN,NaN,125.95,NaN,NaN,NaN,106.75,NaN,NaN,NaN,88.10,NaN,NaN,NaN,70.35,NaN,NaN,NaN,53.65,NaN,NaN,NaN,38.60,NaN,NaN,NaN,25.70,NaN,NaN,NaN,15.70,NaN,NaN,NaN,7.95,NaN,NaN,NaN,3.8,NaN,NaN,NaN,1525.0,NaN,NaN,484.11,0.11,25.9.2009


In [14]:
data1 = data.drop(data.columns[50], axis="columns")
data1.head()

,2009-09-21 00:00:00,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570,Unnamed: 48,Unnamed: 49
,<int64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,86,152.20,NaN,NaN,NaN,132.60,NaN,NaN,NaN,113.20,NaN,NaN,NaN,94.55,NaN,NaN,NaN,76.45,NaN,NaN,NaN,59.25,NaN,NaN,NaN,43.60,NaN,NaN,NaN,29.80,NaN,NaN,NaN,18.65,NaN,NaN,NaN,10.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491.34,0.11
1,85,155.05,NaN,NaN,NaN,135.35,NaN,NaN,NaN,115.90,NaN,NaN,NaN,96.90,NaN,NaN,NaN,78.75,NaN,NaN,NaN,61.45,NaN,NaN,NaN,45.45,NaN,NaN,NaN,31.40,NaN,NaN,NaN,19.80,NaN,NaN,NaN,11.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494.35,0.11
2,84,152.40,NaN,NaN,NaN,132.70,NaN,NaN,NaN,113.30,NaN,NaN,NaN,94.35,NaN,NaN,NaN,76.15,NaN,NaN,NaN,59.05,NaN,NaN,NaN,43.40,NaN,NaN,NaN,29.55,NaN,NaN,NaN,18.45,NaN,NaN,NaN,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490.36,0.11
3,83,147.00,NaN,NaN,NaN,127.25,NaN,NaN,NaN,108.05,NaN,NaN,NaN,89.70,NaN,NaN,NaN,71.95,NaN,NaN,NaN,55.25,NaN,NaN,NaN,39.70,NaN,NaN,NaN,26.95,NaN,NaN,NaN,16.45,NaN,NaN,NaN,8.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,486.99,0.11
4,82,145.55,NaN,NaN,NaN,125.95,NaN,NaN,NaN,106.75,NaN,NaN,NaN,88.10,NaN,NaN,NaN,70.35,NaN,NaN,NaN,53.65,NaN,NaN,NaN,38.60,NaN,NaN,NaN,25.70,NaN,NaN,NaN,15.70,NaN,NaN,NaN,7.95,NaN,NaN,NaN,3.8,NaN,NaN,NaN,1525.0,NaN,NaN,484.11,0.11


In [15]:
data1 = data1.rename(columns={data.columns[0]:'T', data.columns[48]:'S', data.columns[49]:'r'})
data1.head()

,T,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570,S,r
,<int64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>
0,86,152.20,NaN,NaN,NaN,132.60,NaN,NaN,NaN,113.20,NaN,NaN,NaN,94.55,NaN,NaN,NaN,76.45,NaN,NaN,NaN,59.25,NaN,NaN,NaN,43.60,NaN,NaN,NaN,29.80,NaN,NaN,NaN,18.65,NaN,NaN,NaN,10.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491.34,0.11
1,85,155.05,NaN,NaN,NaN,135.35,NaN,NaN,NaN,115.90,NaN,NaN,NaN,96.90,NaN,NaN,NaN,78.75,NaN,NaN,NaN,61.45,NaN,NaN,NaN,45.45,NaN,NaN,NaN,31.40,NaN,NaN,NaN,19.80,NaN,NaN,NaN,11.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494.35,0.11
2,84,152.40,NaN,NaN,NaN,132.70,NaN,NaN,NaN,113.30,NaN,NaN,NaN,94.35,NaN,NaN,NaN,76.15,NaN,NaN,NaN,59.05,NaN,NaN,NaN,43.40,NaN,NaN,NaN,29.55,NaN,NaN,NaN,18.45,NaN,NaN,NaN,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490.36,0.11
3,83,147.00,NaN,NaN,NaN,127.25,NaN,NaN,NaN,108.05,NaN,NaN,NaN,89.70,NaN,NaN,NaN,71.95,NaN,NaN,NaN,55.25,NaN,NaN,NaN,39.70,NaN,NaN,NaN,26.95,NaN,NaN,NaN,16.45,NaN,NaN,NaN,8.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,486.99,0.11
4,82,145.55,NaN,NaN,NaN,125.95,NaN,NaN,NaN,106.75,NaN,NaN,NaN,88.10,NaN,NaN,NaN,70.35,NaN,NaN,NaN,53.65,NaN,NaN,NaN,38.60,NaN,NaN,NaN,25.70,NaN,NaN,NaN,15.70,NaN,NaN,NaN,7.95,NaN,NaN,NaN,3.8,NaN,NaN,NaN,1525.0,NaN,NaN,484.11,0.11


In [16]:
data1 = data1.melt(id_vars=['T', 'r', 'S'], var_name="E").dropna()
data1.rename({'value':'Cobs'}, axis='columns', inplace=True)
data1['T'] = data1['T']/252
data1['S'] = data1['S']/1000
data1['r'] = data1['r']/100
data1['E'] = data1['E']/1000
data1['Cobs'] = data1['Cobs']/1000
data1.head()

,T,r,S,E,Cobs
,<float64>,<float64>,<float64>,<object>,<float64>
0,0.341270,0.0011,0.49134,0.34,0.15220
1,0.337302,0.0011,0.49435,0.34,0.15505
2,0.333333,0.0011,0.49036,0.34,0.15240
3,0.329365,0.0011,0.48699,0.34,0.14700
4,0.325397,0.0011,0.48411,0.34,0.14555


In [22]:
data1.describe()

,T,r,S,Cobs
,<float64>,<float64>,<float64>,<float64>
count,2712.000000,2712.000000,2712.000000,2712.000000
mean,0.132427,0.000606,0.511189,0.083719
std,0.084713,0.000200,0.011638,0.188942
min,0.007937,0.000300,0.475480,0.000010
25%,0.063492,0.000400,0.507030,0.012600
50%,0.119048,0.000600,0.513890,0.061100
75%,0.190476,0.000700,0.518160,0.117953
max,0.341270,0.001200,0.529590,3.125000


In [53]:
from pyfinance.options import BSM
import numpy as np

In [65]:
data0 = data1.iloc[41]
data0
bsm = BSM(data0['S'], data0['E'], data0['T'], data0['r'], 100)

In [66]:
bsm.implied_vol(data0['Cobs'])

0.40364643763147495

In [67]:
bsm.delta()

1.0